In [4]:
%matplotlib ipympl
from impedance.preprocessing import readBioLogic
from impedance.models.circuits import CustomCircuit
import matplotlib.pyplot as plt
import numpy as np
from models import *
import mpmath as mp
import os

In [15]:
folder_path=os.path.join("..","exp data","two combined")
for file_name in os.listdir(folder_path):
    file_path=os.path.join(folder_path,file_name)
    f,z=readBioLogic(file_path)
    
    Rsep=100
    R1=1000
    R2=500
    Q=1e-4
    delta=0.4
    alpha=1
    c=CustomCircuit(initial_guess=[Rsep,R1,R1*Q,R2*Q,delta,alpha],circuit="R_1-TLMtwo")

    mp.dps=400
    c.fit(impedance=z,frequencies=f,bounds=((0,0,0,0,0,0.9),(np.inf,np.inf,np.inf,np.inf,1,1)))
    
    z_fitted=c.predict(frequencies=f)
    print(c)


Circuit string: R_1-TLMtwo
Fit: True

Initial guesses:
    R_1 = 1.00e+02 [Ohm]
  TLMtwo_0 = 1.00e+03 [Ohm]
  TLMtwo_1 = 1.00e-01 [Ohm F]
  TLMtwo_2 = 5.00e-02 [Ohm F]
  TLMtwo_3 = 4.00e-01 []
  TLMtwo_4 = 1.00e+00 []

Fit parameters:
    R_1 = 1.60e+02  (+/- 2.13e-01) [Ohm]
  TLMtwo_0 = 6.76e+02  (+/- 1.34e+00) [Ohm]
  TLMtwo_1 = 4.30e+00  (+/- 1.00e-02) [Ohm F]
  TLMtwo_2 = 1.39e+02  (+/- 9.60e+01) [Ohm F]
  TLMtwo_3 = 9.09e-01  (+/- 2.20e-02) []
  TLMtwo_4 = 9.70e-01  (+/- 1.91e-04) []


Circuit string: R_1-TLMtwo
Fit: True

Initial guesses:
    R_1 = 1.00e+02 [Ohm]
  TLMtwo_0 = 1.00e+03 [Ohm]
  TLMtwo_1 = 1.00e-01 [Ohm F]
  TLMtwo_2 = 5.00e-02 [Ohm F]
  TLMtwo_3 = 4.00e-01 []
  TLMtwo_4 = 1.00e+00 []

Fit parameters:
    R_1 = 1.74e+02  (+/- 3.49e-01) [Ohm]
  TLMtwo_0 = 7.51e+02  (+/- 2.19e+00) [Ohm]
  TLMtwo_1 = 4.91e+00  (+/- 1.82e-02) [Ohm F]
  TLMtwo_2 = 1.99e+03  (+/- 3.62e+03) [Ohm F]
  TLMtwo_3 = 9.69e-01  (+/- 1.87e-02) []
  TLMtwo_4 = 9.72e-01  (+/- 3.68e-04) []


Circuit